In [2]:
import pandas as pd
from getlyrics import az, lcom_artist_searcher
from queue import Queue

In [9]:
tracks_df = pd.read_pickle("./tracks.pkl") 
tracks_df.size

for index, row in tracks_df.iterrows():
    print(tracks_df.loc[index])
    break
    
    

title                       New Bottega
artists    [Torren Foot, Azealia Banks]
lyrics                              NaN
Name: 0, dtype: object


In [ ]:
# Initalise queue and threading
q = Queue()
thread = threading.Thread(target=az, args=())

for index, row in tracks_df.iterrows():
    artists = row['artists']
    title = row['title']

    flag = False
    # Search for each listed artist of song supplied by spotyify on lyrics.com 
    for artist in artists:
        
        #retrieve list of songs by that artist
        song_list = lcom_artist_searcher(artist)
        
        
        # if that artist not found on lyrics.com, first search for the next one (if there is one), then pass it on to azlyrics
        if song_list == 0:
            if flag == False:
                flag = True
                continue

            for i in range(count):
                q.put(row)
            
            if not thread.is_alive():
                thread = threading.Thread(target=az, args=())
                thread.start()
            
        #otherwise,
        else:
            flag = False
            for song in song_list:
                #attempts to match to a song from the retrieved song list 
                if textdistance.jaro_winkler(title, song[0]) >= 0.9:
                    match = True

                    lyrics = lcom_lyrics(song[1])
                    #If the page is empty of lyrics put in AZlyrics queue
                    if lyrics == None:
                        q.put(row)
                        break
                    #If another exception is raised (HTTP error), put it in queue
                    elif lyrics == 0:
                        q.put(row)
                        break
                    else:
                        tracks_df.loc[index]['lyrics'] = lyrics
                    break

            #if no match for the song is found on lyrics.com, query azlyrics instead
            if match == False:
                q.put(row)
                if not thread.is_alive():
                    thread = threading.Thread(target=az, args=())
                    thread.start()

                